In [1]:
import sqlite3, csv, os, math, time
from datetime import datetime
import numpy as np
from scipy.spatial.distance import cdist, pdist
from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn import linear_model
import scipy.io
from itertools import product
import math
import multiprocessing
from functools import partial
os.system("taskset -p 0xff %d" % os.getpid())
import sys
import argparse
from joblib import Parallel, delayed


In [2]:
def parse_args(argv):
    print 'in parse args'
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_fname", default='Filtered.QCed.NoTcorr.SegExclude.HighDAPI.log2.Median.Norm.Cells.csv',
                       help='biomarker csv file')
    parser.add_argument("--data_path", default="/home/lun5/Documents/multiplex/data/",
                       help='directory of spots.db')
    parser.add_argument("--input_path", default="/home/lun5/Documents/multiplex/input/",
                       help='directory of biomarker csv file')
    parser.add_argument("--output_path", default="/home/lun5/Documents/output/",
                       help='directory of output file')
    return parser.parse_args(argv)

In [3]:
def read_data(input_path, data_fname, data_path):
    f = open(os.path.join(input_path,data_fname),'rb')
    f_csv = csv.DictReader(f)
    # load data
    biomarkers = np.load(os.path.join(data_path,'biomarkers.npy'))
    # get biomarker names from csv file
    fields = f_csv.fieldnames[3:15] + f_csv.fieldnames[16:]
    # only take nuclear biomarkers
    nuc_bio_indx = ['.Nuc.' in fields[i] for i in xrange(len(fields))]
    split_name = [str.split(fields[i],'.')[-1] for i in xrange(len(fields))]
    nuc_bio_indx = np.nonzero(np.array(nuc_bio_indx))[0]
    nuc_biomarkers_names = [split_name[nuc_bio_indx[i]] for i in xrange(len(nuc_bio_indx))]
    # only take high quality biomarkers
    f_quality = open(os.path.join(input_path,'biomarkers_quality.csv'),'rb')
    f_csv_quality = csv.DictReader(f_quality)
    marker = []
    quality = []
    for row in f_csv_quality:
        marker.append(row['marker'])
        quality.append(row['quality'])
    # compare lower case names
    nuc_biomarkers_names = [nuc_biomarkers_names[i].lower() for i in xrange(len(nuc_biomarkers_names))]
    marker = [marker[i].lower() for i in xrange(len(marker))]
    nuc_biomarker_qual = np.array([int(quality[marker.index(nuc_biomarkers_names[i])]) 
                               for i in xrange(len(nuc_bio_indx))])
    indx_good_nuc_biomarker = nuc_bio_indx[nuc_biomarker_qual == 1]
    
    # delete biomarkers with too high variance
    delete_bm_indx = [12,19,24,27]
    delete_bm = [str.split(fields[nuc_bio_indx[i]],'.')[-1] for i in delete_bm_indx]
    good_nuc_biomarkers = [split_name[indx_good_nuc_biomarker[i]] for i in xrange(len(indx_good_nuc_biomarker))]
    indx_delete = [good_nuc_biomarkers.index(delete_bm[i]) for i in xrange(1,len(delete_bm))]
    indx_good_nuc_biomarker = np.delete(indx_good_nuc_biomarker,indx_delete)
    good_nuc_biomarkers = [split_name[indx_good_nuc_biomarker[i]] for i in xrange(len(indx_good_nuc_biomarker))]
    bm_data = biomarkers[:,indx_good_nuc_biomarker]
    meta_data = biomarkers[:,[0,1,2,7,8]]
    
    # read ingested spot
    sqlite_file = os.path.join(data_path, 'spots.db')
    conn = sqlite3.connect(sqlite_file)
    conn.text_factory = str
    c = conn.cursor()
    sel_cmd = ('SELECT {tn1}.{coi1} '
           +' FROM {tn1} JOIN {tn2} ON {tn1}.{coi1} = {tn2}.{coi1}'
           + ' WHERE {tn2}.{coi2} == {0}').format(1, 
           coi1='spot_id',coi2='ingested', tn1='spots',tn2 = 'ingestion')
    c.execute(sel_cmd)
    results = c.fetchall()
    ingested_spot_id =  np.asarray([results[i][0] for i in xrange(len(results))])
    
    return (bm_data, meta_data, ingested_spot_id, conn)

In [163]:
def calculate_spot_features(spot_id,all_input, all_params):
    bm_data, meta_data, db_path, NN_OUTPUT, scramble_seed = all_input
    alpha, radius, l1_ratio = all_params
    
    conn = sqlite3.connect(db_path)
    conn.text_factory = str

    c = conn.cursor()
    sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
           coi1='cell_id',coi2='spot_id', tn1='cells')
    
    c.execute(sel_cmd)
    results = c.fetchall()
    cell_id =  np.asarray([results[i][0] for i in xrange(len(results))])
    spot_bm = bm_data[cell_id - 1, :] # biomarker values of the spot            
    spot_meta = meta_data[cell_id -1, :] # x,y coordinates of each cell
    
    sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
           coi1='spot_name',coi2='spot_id', tn1='spots')
    
    c.execute(sel_cmd)
    results = c.fetchall()
    spot_name =  results[0][0]
    
    # get nearest neighbors
    spot_xy = spot_meta[:,:2]
    tree = KDTree(spot_xy)
    ind, dist = tree.query_radius(spot_xy, r = radius, return_distance = True)
    
    # if scramble
    if scramble_seed:
        np.random.seed(scramble_seed)
        spot_bm = spot_bm[np.random.permutation(np.arange(len(cell_id))),:]
        output_name = os.path.join(NN_OUTPUT,spot_name +  '_seed_' + str(scramble_seed) + '.mat')
    else:
        output_name = os.path.join(NN_OUTPUT,spot_name + '.mat')    
    
#    if os.path.isfile(output_name):
#        return 1
    
    #num_non_zeros_nn = [] # number of contributing neighbors
    entropies = [] # entropy
    angle_std = [] # std
    neighbor_id = [] # id of neighbor
    residuals = [] # residuals at each cells
    coeff_neighbors = [] # coefficient of neighbor each cell
    pmf_neighbors = []
    
    for j in xrange(spot_bm.shape[0]):
        clf = linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        
        cell_bm = spot_bm[j,:].reshape(1,-1) # cell of question
        cell_xy = spot_xy[j,:]
        
        nb_bm = spot_bm[ind[j][dist[j]!=0], :] # eliminate itself
        nb_xy = spot_xy[ind[j][dist[j]!=0], :]
        
        curr_neigh_ind = ind[j][dist[j]!=0]
        neighbor_id.append(curr_neigh_ind)      

        if np.min(nb_bm.shape) > 0:
            clf.fit(nb_bm.T,cell_bm.T)
            pred = clf.predict(nb_bm.T).reshape(1,-1)
            resid_biomarkers = np.square(cell_bm - pred)
            residuals.append(np.sqrt(resid_biomarkers.sum())/cell_bm.shape[1])
            coeff_neighbors.append(clf.coef_)
            
            nb_diff_xy = nb_xy - cell_xy
            nb_diff_xy = nb_diff_xy/np.linalg.norm(nb_diff_xy,axis = 1).reshape(-1,1) # normalize the vector
            
            if len(clf.coef_) >= 2: # and (np.linalg.norm(clf.coef_) > 0):
                nb_angles = [0]
                for k in xrange(len(clf.coef_)-1):
                    a = np.dot(nb_diff_xy[0,:],nb_diff_xy[k+1,:])
                    a = np.sign(a)*np.minimum(1,np.abs(a))
                    nb_angles.append(math.acos(a))
                    #nb_angles.append(math.acos(np.dot(nb_diff_xy[0,:],nb_diff_xy[k+1,:])))
                    
                #nb_angles = [0]+[math.acos(np.dot(nb_diff_xy[0,:],nb_diff_xy[i,:])) for i in range(1,nb_diff_xy.shape[0])]
                #print 'Done'
                nb_angles = [(nb_angles[i] + np.pi/100) % 2*np.pi for i in xrange(len(nb_angles))]
                bin_vec = np.linspace(0, 2*np.pi, num=36)
                nb_bin = [(bin_vec -nb_angles[i] >=0).nonzero()[0][0] for i in range(len(nb_angles))]
                pk = np.zeros(36)
                for i in xrange(len(nb_angles)):
                    pk[nb_bin[i]] = pk[nb_bin[i]] + np.abs(clf.coef_[i])
                
                pmf_neighbors.append(np.array(pk))
                cell_entropy = scipy.stats.entropy(pk + 1e-6)
                if np.isinf(cell_entropy):
                    mat_dict={'passed': 0, 'spot_meta':spot_meta,'entropies':entropies,
                            'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
                            'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id
                           }
                    scipy.io.savemat(os.path.join(NN_OUTPUT,spot_name + '.mat'), mat_dict)                    
                    return 0
                entropies.append(cell_entropy)
                #angle_std.append(scipy.stats.circstd(nb_angles))
                coef_ = np.abs(clf.coef_)/np.sum(np.abs(clf.coef_))
                cos_mean_resultant = np.sum([coef_[i]*math.cos(nb_angles[i]) for i in range(len(coef_))])
                sin_mean_resultant = np.sum([coef_[i]*math.sin(nb_angles[i]) for i in range(len(coef_))])
                mean_resultant = np.sqrt(cos_mean_resultant**2+sin_mean_resultant**2)
                cell_angle_std = np.sqrt(2*(1-mean_resultant))
                if np.isnan(cell_angle_std):
                    mat_dict={'passed': 0, 'spot_meta':spot_meta,'entropies':entropies,
                            'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
                            'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id
                           }
                    scipy.io.savemat(output_name, mat_dict)                    
                    return 0
                angle_std.append(cell_angle_std)
            else:
                pmf_neighbors.append(np.array([0]))
                entropies.append(0)
                angle_std.append(0)
        else:
            #print spot_name, j, nb_bm.shape                   
            residuals.append(0)
            entropies.append(0)
            angle_std.append(0)
            pmf_neighbors.append([])
            coeff_neighbors.append([])
    
    mat_dict={'passed': 1, 'spot_meta':spot_meta,'entropies':entropies,
              'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
              'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id}

    
    print output_name
    #print mat_dict
    #scipy.io.savemat(output_name, mat_dict) 
    scipy.io.savemat(output_name, mdict = {'entropies':entropies}) 
    scipy.io.savemat(output_name, mdict = {'residuals':residuals}) 
    scipy.io.savemat(output_name, mdict = {'angle_std':angle_std}) 
    scipy.io.savemat(output_name, mdict = {'pmf':pmf_neighbors}) 
    #print pmf_neighbors
    scipy.io.savemat(output_name, mdict = {'coeff_neigh':coeff_neighbors}) 
    scipy.io.savemat(output_name, mdict = {'neighbor_id':neighbor_id}) 
    
    return 1

In [7]:
def calculate_spot_features(zip_input,all_input, all_params):
    bm_data, meta_data, NN_OUTPUT = all_input
    alpha, radius, l1_ratio, scramble_seed = all_params
    
    spot_id, spot_name, cell_id = zip_input
    
    spot_bm = bm_data[cell_id - 1, :] # biomarker values of the spot            
    spot_meta = meta_data[cell_id -1, :] # x,y coordinates of each cell
    
    # get nearest neighbors
    spot_xy = spot_meta[:,:2]
    tree = KDTree(spot_xy)
    ind, dist = tree.query_radius(spot_xy, r = radius, return_distance = True)
    
    # if scramble
    if scramble_seed:
        np.random.seed(scramble_seed)
        spot_bm = spot_bm[np.random.permutation(np.arange(len(cell_id))),:]
        output_name = os.path.join(NN_OUTPUT,spot_name +  '_seed_' + str(scramble_seed) + '.mat')
    else:
        output_name = os.path.join(NN_OUTPUT,spot_name + '.mat')    
    
#    if os.path.isfile(output_name):
#        return 1
    
    #num_non_zeros_nn = [] # number of contributing neighbors
    entropies = [] # entropy
    angle_std = [] # std
    neighbor_id = [] # id of neighbor
    residuals = [] # residuals at each cells
    coeff_neighbors = [] # coefficient of neighbor each cell
    pmf_neighbors = []
    
    for j in xrange(spot_bm.shape[0]):
        clf = linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        
        cell_bm = spot_bm[j,:].reshape(1,-1) # cell of question
        cell_xy = spot_xy[j,:]
        
        nb_bm = spot_bm[ind[j][dist[j]!=0], :] # eliminate itself
        nb_xy = spot_xy[ind[j][dist[j]!=0], :]
        
        curr_neigh_ind = ind[j][dist[j]!=0]
        neighbor_id.append(curr_neigh_ind)      

        if np.min(nb_bm.shape) > 0:
            clf.fit(nb_bm.T,cell_bm.T)
            pred = clf.predict(nb_bm.T).reshape(1,-1)
            resid_biomarkers = np.square(cell_bm - pred)
            residuals.append(np.sqrt(resid_biomarkers.sum())/cell_bm.shape[1])
            coeff_neighbors.append(clf.coef_)
            
            nb_diff_xy = nb_xy - cell_xy
            nb_diff_xy = nb_diff_xy/np.linalg.norm(nb_diff_xy,axis = 1).reshape(-1,1) # normalize the vector
            
            if len(clf.coef_) >= 2: # and (np.linalg.norm(clf.coef_) > 0):
                nb_angles = [0]
                for k in xrange(len(clf.coef_)-1):
                    a = np.dot(nb_diff_xy[0,:],nb_diff_xy[k+1,:])
                    a = np.sign(a)*np.minimum(1,np.abs(a))
                    nb_angles.append(math.acos(a))
                    #nb_angles.append(math.acos(np.dot(nb_diff_xy[0,:],nb_diff_xy[k+1,:])))
                    
                #nb_angles = [0]+[math.acos(np.dot(nb_diff_xy[0,:],nb_diff_xy[i,:])) for i in range(1,nb_diff_xy.shape[0])]
                #print 'Done'
                nb_angles = [(nb_angles[i] + np.pi/100) % 2*np.pi for i in xrange(len(nb_angles))]
                bin_vec = np.linspace(0, 2*np.pi, num=36)
                nb_bin = [(bin_vec -nb_angles[i] >=0).nonzero()[0][0] for i in range(len(nb_angles))]
                pk = np.zeros(36)
                for i in xrange(len(nb_angles)):
                    pk[nb_bin[i]] = pk[nb_bin[i]] + np.abs(clf.coef_[i])
                
                pmf_neighbors.append(np.array(pk))
                cell_entropy = scipy.stats.entropy(pk + 1e-6)
                if np.isinf(cell_entropy):
                    mat_dict={'passed': 0, 'spot_meta':spot_meta,'entropies':entropies,
                            'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
                            'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id
                           }
                    scipy.io.savemat(os.path.join(NN_OUTPUT,spot_name + '.mat'), mat_dict)                    
                    return 0
                entropies.append(cell_entropy)
                #angle_std.append(scipy.stats.circstd(nb_angles))
                coef_ = np.abs(clf.coef_)/np.sum(np.abs(clf.coef_))
                cos_mean_resultant = np.sum([coef_[i]*math.cos(nb_angles[i]) for i in range(len(coef_))])
                sin_mean_resultant = np.sum([coef_[i]*math.sin(nb_angles[i]) for i in range(len(coef_))])
                mean_resultant = np.sqrt(cos_mean_resultant**2+sin_mean_resultant**2)
                cell_angle_std = np.sqrt(2*(1-mean_resultant))
                if np.isnan(cell_angle_std):
                    mat_dict={'passed': 0, 'spot_meta':spot_meta,'entropies':entropies,
                            'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
                            'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id
                           }
                    scipy.io.savemat(output_name, mat_dict)                    
                    return 0
                angle_std.append(cell_angle_std)
            else:
                pmf_neighbors.append(np.array([0]))
                entropies.append(0)
                angle_std.append(0)
        else:
            #print spot_name, j, nb_bm.shape                   
            residuals.append(0)
            entropies.append(0)
            angle_std.append(0)
            pmf_neighbors.append([])
            coeff_neighbors.append([])
    
    mat_dict={'passed': 1, 'spot_meta':spot_meta,'entropies':entropies,
              'residuals':residuals,'angle_std':angle_std,'pmf':pmf_neighbors,
              'coeff_neigh':coeff_neighbors, 'neighbor_id':neighbor_id}

    
    print output_name
    #print mat_dict
    #scipy.io.savemat(output_name, mat_dict) 
    scipy.io.savemat(output_name, mdict = {'entropies':entropies}) 
    scipy.io.savemat(output_name, mdict = {'residuals':residuals}) 
    scipy.io.savemat(output_name, mdict = {'angle_std':angle_std}) 
    scipy.io.savemat(output_name, mdict = {'pmf':pmf_neighbors}) 
    #print pmf_neighbors
    scipy.io.savemat(output_name, mdict = {'coeff_neigh':coeff_neighbors}) 
    scipy.io.savemat(output_name, mdict = {'neighbor_id':neighbor_id}) 
    
    return 1

In [9]:
def run_calculation():
#     if argv is None:
#        argv = sys.argv[1:]
#     print argv
#     args = parse_args(argv)
    
#     print args
    
    args = {}
    args['output_path'] = "/home/lun5/Documents/multiplex/output/"
    args['data_fname'] = 'Filtered.QCed.NoTcorr.SegExclude.HighDAPI.log2.Median.Norm.Cells.csv'
    args['input_path'] = "/home/lun5/Documents/multiplex/input/"
    args['data_path'] = "/home/lun5/Documents/multiplex/data/"
    
#     parser.add_argument("--data_fname", default='Filtered.QCed.NoTcorr.SegExclude.HighDAPI.log2.Median.Norm.Cells.csv',
#                        help='biomarker csv file')
#     parser.add_argument("--data_path", default="/home/lun5/Documents/multiplex/data/",
#                        help='directory of spots.db')
#     parser.add_argument("--input_path", default="/home/lun5/Documents/multiplex/input/",
#                        help='directory of biomarker csv file')
#     parser.add_argument("--output_path", default="/home/lun5/Documents/output/",
#                        help='directory of output file')
    
    if not os.path.isdir(args['output_path']):
        os.mkdir(args['output_path'])
    
    # COMBINATIONS OF PARAMETER (WILL BE IN ARGS)
    alphas = [1]
    radii = [50] # Values: 50, 100
    l1_ratios = [0.5] # need to add 0 in there
    
    # READ IN DATA
    bm_data, meta_data, ingested_spot_id, conn = read_data(
        args['input_path'], args['data_fname'], args['data_path'])
    conn.text_factory = str
    c = conn.cursor()
    all_spot_names = []
    all_cell_ids = []
    
    for spot_id in ingested_spot_id:    
        sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
               coi1='cell_id',coi2='spot_id', tn1='cells')

        c.execute(sel_cmd)
        results = c.fetchall()
        all_cell_ids.append(np.asarray([results[i][0] for i in xrange(len(results))]))

        sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
               coi1='spot_name',coi2='spot_id', tn1='spots')

        c.execute(sel_cmd)
        results = c.fetchall()
        all_spot_names.append(results[0][0])
        
    
    zip_input = zip(ingested_spot_id, all_spot_names,all_cell_ids)
    db_path = os.path.join(args['data_path'], 'spots.db')
    for alpha, radius, l1_ratio in product(alphas, radii,l1_ratios):
        NN_OUTPUT = os.path.join(args['output_path'], ('Entropy_radius_' + str(radius)) + '_alpha_' +str(alpha) 
                          + '_L1ratio_' + str(l1_ratio))
        if os.path.isdir(NN_OUTPUT):
            print(('Elastic: Already started calculation for alpha = %0.2f radius = %d, l1_ratio = %0.2f') %(
                alpha, radius, l1_ratio))
        else:
            print(('Elastic: Have not calculated for alpha = %0.2f radius = %d, l1_ratio = %0.2f') %(
                alpha, radius, l1_ratio))
            os.makedirs(NN_OUTPUT)

        start_time = time.time()
        
        partial_spot_features = partial(calculate_spot_features, all_input=(
                    bm_data, meta_data, NN_OUTPUT), all_params = (
                alpha, radius, l1_ratio,[]))
        #feature_results = partial_spot_features(21)
        #print feature_results
        #with Parallel(n_jobs = 4) as parallel:
        #    feature_results = parallel(delayed(partial_spot_features)(spot_id)
        #                              for spot_id in ingested_spot_id)
        
        
#         for spot_id in ingested_spot_id[10:50]:
#            print 'spot ',spot_id
#            feat_results = partial_spot_features(spot_id)
#            print feat_results
        
        pool = multiprocessing.Pool(processes=2)
        os.system("taskset -p 0xff %d" % os.getpid())
        feature_results = pool.map(partial_spot_features, zip_input[:30],20)
        #my_it = (spot_id for spot_id in ingested_spot_id)
        #feature_results = pool.map(partial_spot_features, ingested_spot_id[:8],2000)
        pool.close()
        pool.join()
#         print feature_results
        #non_passed_cases = ingested_spot_id[np.nonzero(np.array(feature_results) == 0)[0]]
        #print 'Non scramble: id of cases not passed are', non_passed_cases
        
        
        # scramble
        
#         for seed in [1,50,200]:
#             partial_spot_features = partial(calculate_spot_features, all_input=(
#                         bm_data, meta_data, db_path, NN_OUTPUT, seed), 
#                                             all_params = (alpha, radius, l1_ratio))
#             #feature_results = partial_spot_features(ingested_spot_id[0])
#             pool = multiprocessing.Pool(processes=6)
#             os.system("taskset -p 0xff %d" % os.getpid())
#             feature_results = pool.map(partial_spot_features, ingested_spot_id[:5], 50)
#             pool.close()
#             pool.join()
#             non_passed_cases = ingested_spot_id[np.nonzero(np.array(feature_results) == 0)[0]]
#             print 'Scramble with seed ', seed, ', id of cases not passed are', non_passed_cases
        
        print('Done with alpha=%0.1f, radius = %d in %0.2f seconds' %(
                alpha,radius, time.time() - start_time))    
        
                #c = conn.cursor()
        #for sp in list(non_passed_cases):
        #    sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(sp, 
        #       coi1='spot_name',coi2='spot_id', tn1='spots')
        #    c.execute(sel_cmd)
        #    results = c.fetchall()
        #    sp_name =  results[i][0]
        #    print('Spot %s did not pass' %(sp_name))
        

In [10]:
run_calculation()

Elastic: Have not calculated for alpha = 1.00 radius = 50, l1_ratio = 0.50
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_228.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_275.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_227.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_274.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_226.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_273.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_215.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_272.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA_260_3_214.mat
/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5/AGA

In [40]:
if __name__ == "__main__":
    sys.exit(main())

in parse args


usage: __main__.py [-h] [--data_fname DATA_FNAME] [--data_path DATA_PATH]
                   [--input_path INPUT_PATH] [--output_path OUTPUT_PATH]
__main__.py: error: unrecognized arguments: -f /run/user/1003/jupyter/kernel-654bdcef-76fa-472a-bbd5-353b873b7531.json


SystemExit: 2

In [4]:
args = {}
args['output_path'] = "/home/lun5/Documents/multiplex/output/"
args['data_fname'] = 'Filtered.QCed.NoTcorr.SegExclude.HighDAPI.log2.Median.Norm.Cells.csv'
args['input_path'] = "/home/lun5/Documents/multiplex/input/"
args['data_path'] = "/home/lun5/Documents/multiplex/data/"
# READ IN DATA
bm_data, meta_data, ingested_spot_id, conn = read_data(
    args['input_path'], args['data_fname'], args['data_path'])

conn.text_factory = str
c = conn.cursor()
all_spot_names = []
all_cell_ids = []

for spot_id in ingested_spot_id:    
    sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
           coi1='cell_id',coi2='spot_id', tn1='cells')

    c.execute(sel_cmd)
    results = c.fetchall()
    all_cell_ids.append(np.asarray([results[i][0] for i in xrange(len(results))]))

    sel_cmd = ('SELECT {coi1} FROM {tn1} WHERE {coi2}=={0}').format(spot_id, 
           coi1='spot_name',coi2='spot_id', tn1='spots')

    c.execute(sel_cmd)
    results = c.fetchall()
    all_spot_names.append(results[0][0])

print len(ingested_spot_id), len(all_cell_ids), len(all_spot_names)


653 653 653


In [5]:
zip_input = zip(ingested_spot_id,all_cell_ids, all_spot_names)
for spot_id, cell_id, spot_name in zip_input:
    print spot_id, spot_name, len(cell_id)

1 AGA_260_3_275 2689
2 AGA_260_3_274 2177
3 AGA_260_3_273 2209
4 AGA_260_3_272 2088
5 AGA_260_3_271 2620
6 AGA_260_3_270 3065
7 AGA_260_3_269 2233
11 AGA_260_3_255 1855
12 AGA_260_3_254 2345
13 AGA_260_3_253 2167
15 AGA_260_3_251 3046
16 AGA_260_3_250 1748
17 AGA_260_3_249 2240
18 AGA_260_3_248 2441
19 AGA_260_3_247 2618
20 AGA_260_3_246 2219
21 AGA_260_3_235 2093
22 AGA_260_3_234 1940
24 AGA_260_3_232 1545
25 AGA_260_3_231 2128
27 AGA_260_3_228 1783
28 AGA_260_3_227 2754
29 AGA_260_3_226 1527
30 AGA_260_3_215 1777
31 AGA_260_3_214 1936
32 AGA_260_3_213 2905
33 AGA_260_3_212 2671
34 AGA_260_3_211 2032
35 AGA_260_3_210 2443
36 AGA_260_3_209 2137
37 AGA_260_3_208 2210
38 AGA_260_3_207 1735
39 AGA_260_3_206 1695
40 AGA_260_3_195 1796
41 AGA_260_3_194 2098
42 AGA_260_3_193 2548
43 AGA_260_3_192 2118
44 AGA_260_3_191 2685
45 AGA_260_3_190 2539
46 AGA_260_3_189 2352
47 AGA_260_3_188 1908
48 AGA_260_3_187 1932
49 AGA_260_3_186 2766
51 AGA_260_3_174 1551
52 AGA_260_3_173 1719
53 AGA_260_3_172 

In [6]:
spot_id, cell_id, spot_name = zip_input[0]

In [13]:
mat_file = os.path.join('/home/lun5/Documents/multiplex/output/Entropy_radius_50_alpha_1_L1ratio_0.5',
                       'AGA_260_3_271.mat')
mat_dict = scipy.io.loadmat(mat_file)

In [17]:
mat_dict['pmf'][0][1]

array([[ 0.        ,  0.15568904,  0.        ,  0.        ,  0.25518656,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.16338062,  0.16930836,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.11267287,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])